In [1]:
rm -rf /kaggle/working/*

In [2]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 6.3 MB/s eta 0:00:00


In [3]:
!git clone --branch main https://github.com/giankev/Ancient-to-Modern-Italian-Automatic-Translation.git

Cloning into 'Ancient-to-Modern-Italian-Automatic-Translation'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 60 (delta 29), reused 36 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (60/60), 57.08 KiB | 2.28 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [4]:
from kaggle_secrets import UserSecretsClient
import huggingface_hub
import os

try:
    user_secrets = UserSecretsClient()
    API_KEY = user_secrets.get_secret("api_google_ai")
    print("API KEY Loaded")
except Exception as e:
    print(f"Error during programmatic login: {e}")
    print("Please ensure your HF_TOKEN secret is correctly set in Kaggle.")

API KEY Loaded


In [5]:
from time import sleep

In [12]:
import pandas as pd

csv_translation = "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/Gemma2b-it-translation/Gemma2b-it-translations_context_learning.csv"
df = pd.read_csv(csv_translation)

In [13]:
df.head()

,old_text,translation
0,quella guerra ben fatta l' opera perché etc. E...,"""Quella guerra ben fatta, l'opera perché etc. ..."
1,"crudele, e di tutte le colpe pigli vendetta, c...","""crudele, e di tutte le colpe si prende vendet..."
2,Non d' altra forza d' animo fue ornato Ponzio ...,"""Non ha altra forza d'animo che l'onore di Pon..."
3,Se questo piace a tutti e se 'l tempo hae biso...,"""Se questo piace a tutti e se il tempo ha biso..."
4,Officio di questa arte pare che sia dicere app...,"""L'arte di questa disciplina sembra essere que..."


In [7]:
from google import genai

client = genai.Client(api_key=API_KEY)

In [13]:
def prompt_evaluation(original: str, translated: str):
    prompt =  f"""Valuta la seguente traduzione dall'italiano antico all'italiano moderno.
                Concentrati sulla fedeltà semantica e la correttezza grammaticale.
                Fornisci un punteggio da 1 a 5 basandoti sulla seguente rubrica:
                1. Traduzione completamente inaccettabile: la traduzione non ha pertinenza con il significato originale, la frase generata è incomprensibile (gibberish) o senza senso.
                2. Errori semantici gravi, omissioni o aggiunte sostanziali rispetto alla frase originale. Gli errori sono di natura semantica e sintattica. È qualcosa che un umano non scriverebbe mai.
                3. Traduzione parzialmente errata: la traduzione è mediocre, contiene errori, ma sono per lo più errori minori, come refusi o piccoli errori semantici.
                4. Buona traduzione: la traduzione è per lo più corretta, sostanzialmente fedele al testo originale, ma lo stile potrebbe non corrispondere perfettamente alla frase originale; è comunque fluente, comprensibile e semanticamente accettabile.
                5. Traduzione perfetta: la traduzione è accurata, fluente, completa e coerente. Ha mantenuto il significato originale il più possibile.
                fornisci solo il numero del punteggio (da 1 a 5).
                frase originale: "{original}"
                frase tradotta: "{translated}"
                
                """
    return prompt

In [9]:
def prompt_evaluation_fluency(original: str, translated: str) -> str:
    prompt = f"""Valuta ESCLUSIVAMENTE la FLUENZA e la NATURALEZZA della seguente traduzione in italiano moderno.
                Ignora per ora la sua precisione semantica rispetto al testo originale arcaico.
                Fornisci un punteggio da 1 a 3 basandoti sulla seguente rubrica:
                
                1. Poco Fluida/Innaturale: La frase moderna è sgrammaticata, difficile da capire, o suona artificiale e non come la scriverebbe un madrelingua.
                2. Abbastanza Fluida: La frase moderna è grammaticalmente comprensibile ma presenta parole usate in modo errato che ne riducono la naturalezza.
                3. Perfettamente Fluida e Naturale: La frase moderna scorre bene, usa un lessico e una sintassi del tutto appropriati per l'italiano moderno e suona naturale.
                
                Fornisci solo il numero del punteggio (da 1 a 3).
                
                frase originale (Italiano Antico): "{original}"
                traduzione proposta (Italiano Moderno da Valutare): "{translated}"
                """
    
    return prompt

In [15]:
results = []
for index, row in df.iterrows():
    
    original = row["old_text"]
    translation = row["translation"]
    prompt = prompt_evaluation_fluency(original, translation)

    response = client.models.generate_content(
             model="gemini-2.0-flash-lite",
             contents=prompt
    )

    results.append({
        "sentence": original,
        "translation": translation,
        "score": response.text
    })

    sleep(2) 

In [16]:
df_results = pd.DataFrame(results)
df_results.to_csv("/kaggle/working/evaluation_Gemma2b-it_fluency.csv", index=False, encoding='utf-8')

In [17]:
import json

with open("/kaggle/working/evaluation_Gemma2b-it_fluency.jsonl", "w", encoding="utf-8") as f:
    for entry in results:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')